In [155]:
import pandas as pd
import numpy as np
import math
import os
from datetime import datetime
import subprocess
import shutil
import time

def get_date_subfolder():
    current_date = datetime.now()
    return current_date.strftime('%Y_%m_%d')

gort_root_path = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in'
root_output_path = '/data/shared/src/STV/NEON_TEAK/allen/figs/veg_input'
plotloc_csv_filename = '/data/shared/src/STV/NEON_TEAK/allen/data/NEON_struct-plant/matching_shapefiles_tiffs.csv'
shapefile_path = "/data/shared/src/arojas/NEON/data/raw/spatial/baseplots/TEAK_baseplots_utm/TEAK_baseplots_utm.shp"
R_generated_csv = '/data/shared/src/STV/NEON_TEAK/allen/data/tree_locations.csv'
orth_tiff_folder_2023 = '/data/shared/rsdata/orthophotos/TEAK/NEON_images-camera-ortho-mosaic/NEON.D17.TEAK.DP3.30010.001.2023-07.basic.20240905T014815Z.PROVISIONAL'
height_cutoffs = "/data/shared/src/STV/NEON_TEAK/allen/data/NEON_struct-plant/TEAK_sites_height_cutoff.csv"

TEAK_plotloc_unique_df = pd.read_csv(plotloc_csv_filename)

date_subfolder = get_date_subfolder()
gort_date_path = os.path.join(gort_root_path, date_subfolder)
date_output_path = os.path.join(root_output_path, date_subfolder)

# Ensure the full save path exists
os.makedirs(gort_date_path, exist_ok=True)

app_indv_100_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_full_100.csv"))
app_indv_400_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_full_400.csv"))

app_indv_100_df = app_indv_100_df.dropna(subset=['height']).sort_values(by='plotID_x')
app_indv_400_df = app_indv_400_df.dropna(subset=['height']).sort_values(by='plotID_x')

app_indv_R_df = pd.read_csv(R_generated_csv)

app_indv_100_locations_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_location_only_100.csv"))
app_indv_400_locations_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_location_only_400.csv"))

gort_input_dir = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/' + date_subfolder
gort_output_dir = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/' + date_subfolder
gort_process_dir = '/data/shared/src/STV/simscenes/LidarSimulation/model/'

In [156]:
def process_gort_inputs(app_indv_locations_df, app_indv_df, TEAK_plotloc_unique_df, save_path, plot_ident):
    # Rename column and sort values
    app_indv_df = app_indv_df.rename(columns={'plotID_x': 'plotID'})
    app_indv_df = app_indv_df.sort_values(by='plotID', ascending=True)
    drop_unknown_df = app_indv_df[app_indv_df['subplotID'].str.contains('unknown') == False]

    # Obtains nique plotIDs count
    unique_plotIDs = drop_unknown_df['plotID'].unique()
    # Declaration of gort_inputs table
    gort_inputs = pd.DataFrame({'plotID': unique_plotIDs, 'Tree Density': np.nan})
    
    # Tree Density Calculation
    for plot_id in unique_plotIDs:
        # For debugging purposes, we keep track of the number of individual growthForms that have and do not have unknowns (in the subplot column)
        # And keeps track of the individuals that do or do not have a relative location entry
        with_unknowns_df = app_indv_df[app_indv_df['plotID'] == plot_id]
        without_unknowns_df = drop_unknown_df[drop_unknown_df['plotID'] == plot_id]
        locations_df = app_indv_locations_df[app_indv_locations_df['plotID_x'] == plot_id]
        subplot_unique_count = len(without_unknowns_df['subplotID'].unique())
        
        tree_density = 0
        
        if plot_ident == '100':
            tree_density = len(without_unknowns_df['subplotID']) / 400
        elif plot_ident == '400':
            tree_density = len(without_unknowns_df['subplotID']) / 800
            
        gort_inputs.loc[gort_inputs['plotID'] == plot_id, 'Tree Density'] = tree_density

        # Debug statements
        print(f"Plot ID: {plot_id}")
        # print(f"Number of trees (with unknowns): {len(with_unknowns_df['subplotID'])}")
        print(f"Number of trees (without unknowns): {len(without_unknowns_df['subplotID'])}")
        # print(f"Number of trees (with locations): {len(locations_df['subplotID'])}")
        print(f"Number of unique subplots (without unknowns): {subplot_unique_count}")
        print(f"Normalized count: {tree_density}")
        print("------")

    gort_inputs = gort_inputs.sort_values(by='plotID', ascending=True)

    noshrub_noheightna_df = app_indv_df[app_indv_df['height'].notna()]
    noshrub_noheightna_df = noshrub_noheightna_df.copy()

    # Used two different methods to initialize column values to np.nan
    if 'Vertical_Crown_Radius' not in noshrub_noheightna_df.columns:
        noshrub_noheightna_df['Vertical_Crown_Radius'] = np.nan
    if 'Crown_Center_Height' not in noshrub_noheightna_df.columns:
        noshrub_noheightna_df.loc[:, 'Crown_Center_Height'] = np.nan

    # Mean Vertical Crown Radius Calculation
    for index, row in noshrub_noheightna_df.iterrows():
        vert_crown_radius = (row['height'] - row['baseCrownHeight']) / 2

        noshrub_noheightna_df.at[index, 'Vertical_Crown_Radius'] = vert_crown_radius
        noshrub_noheightna_df.at[index, 'Crown_Center_Height'] = row['height'] - vert_crown_radius

    mean_vert_crown_radius = noshrub_noheightna_df.groupby('plotID')['Vertical_Crown_Radius'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_vert_crown_radius, on='plotID', how='left')
    gort_inputs.rename(columns={'Vertical_Crown_Radius': 'Mean_Vert_Crown_Radius'}, inplace=True)

    # Mean/STD Crown Center Height Calculation
    mean_std_crown_center_height = noshrub_noheightna_df.groupby('plotID')['Crown_Center_Height'].agg(['mean', 'std']).reset_index()
    mean_std_crown_center_height.rename(columns={'mean': 'Mean_Crown_Center_Height', 'std': 'Std_Crown_Center_Height'}, inplace=True)
    gort_inputs = pd.merge(gort_inputs, mean_std_crown_center_height, on='plotID', how='left')
    
    # Mean Horizontal Crown Radius Calculation
    mean_crown_radius = noshrub_noheightna_df.groupby('plotID')['crown_radius'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_crown_radius, on='plotID', how='left')
    gort_inputs.rename(columns={'crown_radius': 'Mean_Horz_Crown_Radius'}, inplace=True)

    # Individual and Total Crown Volume Calculation
    if 'Crown_Volume' not in noshrub_noheightna_df.columns:
        noshrub_noheightna_df['Crown_Volume'] = np.nan

    for index, row in noshrub_noheightna_df.iterrows():
        crown_volume = (4/3) * math.pi * (row['crown_radius']**2) * row['Vertical_Crown_Radius']
        noshrub_noheightna_df.at[index, 'Crown_Volume'] = crown_volume

    total_crown_volume = noshrub_noheightna_df.groupby('plotID')['Crown_Volume'].sum().reset_index()
    gort_inputs = pd.merge(gort_inputs, total_crown_volume, on='plotID', how='left')
    gort_inputs.rename(columns={'Crown_Volume': 'Total Crown_Volume'}, inplace=True)

    # Mean Crown Volume Calculation
    gort_inputs['Mean_Crown_Volume'] = (4/3) * math.pi * (gort_inputs['Mean_Horz_Crown_Radius'] ** 2) * gort_inputs['Mean_Vert_Crown_Radius']
    
    # Fa Calculation:
    # Merges gort_inputs with LAI results from spectrometer on plot_id
    # merged_df = gort_inputs.merge(TEAK_plotloc_unique_df[['namedLocation', 'LAI_Average_exclude_negatives']], left_on='plotID', right_on='namedLocation', how='left')
    merged_df = gort_inputs.merge(TEAK_plotloc_unique_df[['namedLocation', 'small_footprint_lidar_LAI']], left_on='plotID', right_on='namedLocation', how='left')

    # Calculate the 'Fa'
    # merged_df['Fa'] = merged_df['LAI_Average_exclude_negatives'] / (merged_df['Mean_Crown_Volume'] * merged_df['Tree Density'])
    merged_df['Fa'] = merged_df['small_footprint_lidar_LAI'] / (merged_df['Mean_Crown_Volume'] * merged_df['Tree Density'])

    # gort_inputs['LAI'] = merged_df['LAI_Average_exclude_negatives']
    gort_inputs['LAI'] = merged_df['small_footprint_lidar_LAI']
    gort_inputs['Fa'] = merged_df['Fa']

    # H1, H2 Calculation
    if 'h1' not in gort_inputs.columns:
        gort_inputs['h1'] = np.nan
    if 'h2' not in gort_inputs.columns:
        gort_inputs['h2'] = np.nan

    for index, row in gort_inputs.iterrows():
        mean_crct_height = row['Mean_Crown_Center_Height']
        std_crct_height = row['Std_Crown_Center_Height']
        h1 = mean_crct_height - (2 * std_crct_height)
        h2 = mean_crct_height + (2 * std_crct_height)

        if h1 < row['Mean_Vert_Crown_Radius']:
            h1 = row['Mean_Vert_Crown_Radius']

        gort_inputs.at[index, 'h1'] = h1
        gort_inputs.at[index, 'h2'] = h2

    # Mean Height Calculation + dz values
    mean_height = noshrub_noheightna_df.groupby('plotID')['height'].mean().reset_index()
    gort_inputs = pd.merge(gort_inputs, mean_height, on='plotID', how='left')
    gort_inputs.rename(columns={'height': 'Mean_Height'}, inplace=True)

    gort_inputs['dz'] = 0.1
    gort_inputs = gort_inputs.round(4)

    gort_inputs = gort_inputs[gort_inputs['h1'].notna()]
    filename = "gort_input_values_" + plot_ident + ".csv"
    output_path = os.path.join(save_path, filename)
    gort_inputs.to_csv(output_path, index=False)
    
    print(f"file saved to: {output_path}")

    return gort_inputs

def output_gort_files(gort_inputs, save_path, gort_ident):
    gort_save_path = os.path.join(save_path, gort_ident)
    for index, row in gort_inputs.iterrows():
        output_file_name = os.path.join(gort_save_path, f"{row['plotID']}.in")

        os.makedirs(gort_save_path, exist_ok=True)

        with open(output_file_name, 'w') as file:
            file.write('1\n')
            file.write('\t' + str(row['Tree Density']))
            file.write('\t' + str(row['Fa']))
            file.write('\t' + str(row['h1']))
            file.write('\t' + str(row['h2']))
            file.write('\t' + str(row['dz']))
            file.write('\t' + str(row['Mean_Horz_Crown_Radius']))
            file.write('\t' + str(row['Mean_Vert_Crown_Radius']))
            file.write('\t' + '0.00')
            file.write('\t' + '0.4291')
            file.write('\t' + '0.4239')
            file.write('\t' + '0.35156')
            file.write('\t' + '1')
        
        print(f"file saved to:  {output_file_name}")

In [157]:
print("100m2: \n")
gort_inputs_100_df = process_gort_inputs(app_indv_100_locations_df, app_indv_100_df, TEAK_plotloc_unique_df, date_output_path, "100")
print("400m2: \n")
gort_inputs_400_df = process_gort_inputs(app_indv_400_locations_df, app_indv_400_df, TEAK_plotloc_unique_df, date_output_path, "400")

100m2: 

Plot ID: TEAK_001
Number of trees (without unknowns): 11
Number of unique subplots (without unknowns): 1
Normalized count: 0.0275
------
Plot ID: TEAK_002
Number of trees (without unknowns): 14
Number of unique subplots (without unknowns): 4
Normalized count: 0.035
------
Plot ID: TEAK_003
Number of trees (without unknowns): 17
Number of unique subplots (without unknowns): 3
Normalized count: 0.0425
------
Plot ID: TEAK_005
Number of trees (without unknowns): 10
Number of unique subplots (without unknowns): 2
Normalized count: 0.025
------
Plot ID: TEAK_006
Number of trees (without unknowns): 12
Number of unique subplots (without unknowns): 3
Normalized count: 0.03
------
Plot ID: TEAK_007
Number of trees (without unknowns): 12
Number of unique subplots (without unknowns): 4
Normalized count: 0.03
------
Plot ID: TEAK_010
Number of trees (without unknowns): 4
Number of unique subplots (without unknowns): 3
Normalized count: 0.01
------
Plot ID: TEAK_011
Number of trees (withou

In [158]:
output_gort_files(gort_inputs_100_df, gort_date_path, "Gort_100")
output_gort_files(gort_inputs_400_df, gort_date_path, "Gort_400")

file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_001.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_002.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_003.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_005.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_006.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_007.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_010.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_011.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_012.in
file saved to:  /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_013.in
file saved to:  /dat

In [159]:
def run_gort_teak(input_dir, output_dir, process_dir):
    date_subfolder = get_date_subfolder()

    # Find all .in files recursively in the input directory
    fplot = [os.path.join(root, file)
             for root, _, files in os.walk(input_dir)
             for file in files if file.endswith('.in')]
    
    # Get the current working directory at the start of the script
    initial_dir = os.getcwd()

    # Process each .in file
    for infile in fplot:
        # Get the relative path of the infile with respect to the input directory
        rel_path = os.path.relpath(infile, input_dir)
        
        # Determine the corresponding output file path
        out_file = os.path.join(output_dir, rel_path).replace('.in', '.out')
        
        # Ensure the directory for the output file exists
        os.makedirs(os.path.dirname(out_file), exist_ok=True)
        
        # Copy the input file to gort.in in the process directory
        process_infile = os.path.join(process_dir, 'gort.in')
        shutil.copy(infile, process_infile)
        print(f"Copied {infile} to {process_infile}")
        
        # Change to the process directory
        os.chdir(process_dir)
        
        # Run the Fortran script in the process directory
        cmd_run = './gort_lidar > temp.txt'
        result = subprocess.run(cmd_run, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"Error running GORT model: {result.stderr}")
            break

        # Wait for a short time to ensure gort.out is fully generated
        # time.sleep(1)  # Delay in seconds; adjust as needed
        
        # Move gort.out from the process directory to the corresponding output file path
        process_outfile = os.path.join(process_dir, 'gort.out')
        shutil.move(process_outfile, out_file)
        print(f'Moved {process_outfile} to {out_file}')
        
        # Change back to the initial directory
        os.chdir(initial_dir)

In [160]:
run_gort_teak(gort_input_dir, gort_output_dir, gort_process_dir)

Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_001.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_001.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_002.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_002.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_003.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_003.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100

Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_007.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_010.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_010.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_011.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_011.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_012.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data

In [ ]:
def group_trees_by_height(app_indv_df, cutoffs_df):
    # Filter to only include rows where 'height' is not NaN
    app_indv_filtered = app_indv_df[app_indv_df['height'].notna()].copy()

    # Get unique plotIDs from app_indv_df
    unique_plot_ids = app_indv_filtered['plotID_x'].unique()

    # Filter cutoffs_df to only include rows where plotID is in app_indv_df
    cutoffs_df_filtered = cutoffs_df[cutoffs_df['plotid'].isin(unique_plot_ids)]

    # Create an empty DataFrame to store grouped data
    grouped_df = pd.DataFrame()

    # Iterate over each plotid in the cutoff DataFrame
    for plot_id in cutoffs_df_filtered['plotid'].unique():
        # Get the height cutoffs for the current plot
        height_cutoffs = cutoffs_df_filtered.loc[cutoffs_df_filtered['plotid'] == plot_id, 'height_cutoff'].values
        
        # Skip if no height_cutoff data exists for this plot_id or if it's labeled as 'No Group'
        if len(height_cutoffs) == 0 or height_cutoffs[0] == 'No Group':
            # If no valid cutoffs exist, assign all trees to 'No Group'
            plot_trees = app_indv_filtered[app_indv_filtered['plotID_x'] == plot_id].copy()
            plot_trees['Group_Label'] = 'No Group'
            grouped_df = pd.concat([grouped_df, plot_trees], ignore_index=True)
            continue  # Skip to next plot

        # Parse the height cutoffs (split by comma if needed)
        try:
            height_cutoffs = sorted([float(x) for x in str(height_cutoffs[0]).split(',')])
        except ValueError:
            print(f"Warning: Invalid cutoff values for plot {plot_id}. Assigning 'No Group' label to all trees.")
            plot_trees = app_indv_filtered[app_indv_filtered['plotID_x'] == plot_id].copy()
            plot_trees['Group_Label'] = 'No Group'
            grouped_df = pd.concat([grouped_df, plot_trees], ignore_index=True)
            continue

        # Filter app_indv_df for trees in the current plotID
        plot_trees = app_indv_filtered[app_indv_filtered['plotID_x'] == plot_id].copy()
        
        # Debug statement: Total number of trees in the plot
        total_trees = len(plot_trees)
        print(f"Total number of trees in plot {plot_id}: {total_trees}")

        # Initialize a variable to track the total number of trees grouped
        grouped_trees_count = 0
        
        # Iterate over the cutoffs and create groups
        for i, cutoff in enumerate(height_cutoffs):
            if i == 0:
                # First group: trees below the first cutoff
                below_cutoff_df = plot_trees[plot_trees['height'] < cutoff].copy()
                below_cutoff_df['Group_Label'] = f"Below_{cutoff}"
                grouped_df = pd.concat([grouped_df, below_cutoff_df], ignore_index=True)
                
                # Debug statement: Number of trees in this group
                print(f"  Number of trees below {cutoff}: {len(below_cutoff_df)}")
                grouped_trees_count += len(below_cutoff_df)

            if i < len(height_cutoffs) - 1:
                # Middle groups: trees between two cutoffs
                next_cutoff = height_cutoffs[i + 1]
                between_cutoffs_df = plot_trees[(plot_trees['height'] >= cutoff) & (plot_trees['height'] < next_cutoff)].copy()
                between_cutoffs_df['Group_Label'] = f"{cutoff}_to_{next_cutoff}"
                grouped_df = pd.concat([grouped_df, between_cutoffs_df], ignore_index=True)
                
                # Debug statement: Number of trees in this group
                print(f"  Number of trees between {cutoff} and {next_cutoff}: {len(between_cutoffs_df)}")
                grouped_trees_count += len(between_cutoffs_df)
            else:
                # Last group: trees above the last cutoff
                above_cutoff_df = plot_trees[plot_trees['height'] >= cutoff].copy()
                above_cutoff_df['Group_Label'] = f"Above_{cutoff}"
                grouped_df = pd.concat([grouped_df, above_cutoff_df], ignore_index=True)

                # Debug statement: Number of trees in this group
                print(f"  Number of trees above {cutoff}: {len(above_cutoff_df)}")
                grouped_trees_count += len(above_cutoff_df)
                
        # Debug statement: Check if total number of trees matches the sum of grouped trees
        if total_trees == grouped_trees_count:
            print(f"  Matched: {grouped_trees_count}")
        else:
            print(f"  Mismatch! Expected {total_trees} but grouped {grouped_trees_count} trees.")

    return grouped_df

def calculate_gort_inputs(app_indv_df, TEAK_plotloc_unique_df, plot_ident, grouped_trees_df):
    app_indv_df = app_indv_df.rename(columns={'plotID_x': 'plotID'}).sort_values(by='plotID', ascending=True)
    drop_unknown_df = app_indv_df[~app_indv_df['subplotID'].str.contains('unknown')]

    unique_plotIDs = drop_unknown_df['plotID'].unique()
    gort_inputs = pd.DataFrame({'plot_id': unique_plotIDs, 'Tree Density': np.nan})
    
    for plot_id in unique_plotIDs:
        without_unknowns_df = drop_unknown_df[drop_unknown_df['plotID'] == plot_id]
        # Determine plot area based on plot_ident
        if plot_ident == '100':
            plot_area = 400
        elif plot_ident == '400':
            plot_area = 800
        # Add plot_area to gort_inputs DataFrame
        gort_inputs.loc[gort_inputs['plot_id'] == plot_id, 'plot_area'] = plot_area

        # Calculate Tree Density (number of trees divided by plot area)
        tree_density = len(without_unknowns_df) / plot_area
        gort_inputs.loc[gort_inputs['plot_id'] == plot_id, 'Tree Density'] = tree_density

    grouped_trees_df['Vertical_Crown_Radius'] = (grouped_trees_df['height'] - grouped_trees_df['baseCrownHeight']) / 2
    grouped_trees_df['Crown_Center_Height'] = grouped_trees_df['height'] - grouped_trees_df['Vertical_Crown_Radius']
    grouped_trees_df['Crown_Volume'] = (4 / 3) * math.pi * (grouped_trees_df['crown_radius'] ** 2) * grouped_trees_df['Vertical_Crown_Radius']

    # Process grouped trees for crown radius and volume calculations
    all_trees_data = []
    for (plot_id, group_label), trees in grouped_trees_df.groupby(['plotID_x', 'Group_Label']):
        trees = trees.copy()
        
        tree_density_group = len(trees) / plot_area
        mean_vert_crown_radius = trees['Vertical_Crown_Radius'].mean()
        mean_crown_radius = trees['crown_radius'].mean()
        total_crown_volume = trees['Crown_Volume'].sum()
        mean_crown_volume = (4/3) * math.pi * (mean_crown_radius ** 2) * mean_vert_crown_radius
        mean_crown_center_height = trees['Crown_Center_Height'].mean()

        # Check for any NaN values in key columns
        if trees['Crown_Center_Height'].isna().sum() > 0:
            print(f"NaN values found in Crown_Center_Height for plot {plot_id}, group {group_label}")
        if trees['Vertical_Crown_Radius'].isna().sum() > 0:
            print(f"NaN values found in Vertical_Crown_Radius for plot {plot_id}, group {group_label}")
        
        # Calculate h1 and h2 values (bounds for crown center height)
        if len(trees) > 1:  # Only calculate standard deviation if there are enough trees
            std_crown_center_height = trees['Crown_Center_Height'].std()
            # h1 = mean_crown_center_height - (2 * std_crown_center_height)
            h1 = max(0, mean_crown_center_height - (2 * std_crown_center_height))
            h2 = mean_crown_center_height + (2 * std_crown_center_height)
        else:
            # For only one tree, set h1 and h2 to the crown center height of that tree, std_crct_height is 0
            std_crown_center_height = 0
            h1 = mean_crown_center_height
            h2 = h1 + 0.1

        # Debugging output to track the calculations for each group
        print(f"Plot ID: {plot_id}, Group: {group_label}")
        print(f"  Mean Vertical Crown Radius: {mean_vert_crown_radius}")
        print(f"  Mean Crown Radius: {mean_crown_radius}")
        print(f"  Total Crown Volume: {total_crown_volume}")
        print(f"  Mean Crown Volume: {mean_crown_volume}")
        print(f"  Mean Crown Center Height: {mean_crown_center_height}")
        print(f"  Std Crown Center Height: {std_crown_center_height}")
        print(f"  h1: {h1}, h2: {h2}")
        
        all_trees_data.append({
            'plot_id': plot_id,
            'group': group_label,
            'Tree_Density_group': tree_density_group,
            'Mean_Vert_Crown_Radius_group': mean_vert_crown_radius,
            'Mean_Crown_Radius_group': mean_crown_radius,
            'Total_Crown_Volume_group': total_crown_volume,
            'Mean_Crown_Volume_group': mean_crown_volume,
            'Mean_Crown_Center_Height_group': mean_crown_center_height,
            'Std_Crown_Center_Height_group': std_crown_center_height,
            'h1_group': h1,
            'h2_group': h2,
        })

    all_trees_df = pd.DataFrame(all_trees_data)
    gort_inputs = gort_inputs.merge(all_trees_df, on='plot_id', how='left')

    # Now calculate plot-level values
    plot_level_data = grouped_trees_df.groupby('plotID_x').agg({
        'crown_radius': 'mean',
        'height': 'mean',
        'baseCrownHeight': 'mean',
        'Crown_Center_Height': 'min'  # Compute minimum Crown Center Height for each plot
    }).reset_index()

    # Plot-level calculations
    plot_level_data['Vertical_Crown_Radius_plot'] = (plot_level_data['height'] - plot_level_data['baseCrownHeight']) / 2
    plot_level_data['Mean_Crown_Radius_plot'] = plot_level_data['crown_radius']
    plot_level_data['Mean_Crown_Volume_plot'] = (4/3) * math.pi * (plot_level_data['Mean_Crown_Radius_plot'] ** 2) * plot_level_data['Vertical_Crown_Radius_plot']
    
    # Replace h1 = 0 values with minimum Crown_Center_Height for each plot
    for plot_id in gort_inputs['plot_id'].unique():
        # Get minimum Crown_Center_Height for the plot
        min_crown_center_height = plot_level_data.loc[plot_level_data['plotID_x'] == plot_id, 'Crown_Center_Height'].values[0]
        # Find rows where h1_group is 0 for the current plot_id
        rows_to_update = gort_inputs[(gort_inputs['plot_id'] == plot_id) & (gort_inputs['h1_group'] == 0)]
        # Set h1 values of 0 to the plot's minimum Crown_Center_Height
        gort_inputs.loc[(gort_inputs['plot_id'] == plot_id) & (gort_inputs['h1_group'] == 0), 'h1_group'] = min_crown_center_height
        # Debugging: Print replacement info
        if not rows_to_update.empty:
            print(f"Plot {plot_id}: Replaced h1_group=0 with min Crown_Center_Height {min_crown_center_height}")

    # Merge only the plot-level mean crown radius and crown volume into gort_inputs
    gort_inputs = gort_inputs.merge(plot_level_data[['plotID_x', 'Vertical_Crown_Radius_plot', 'Mean_Crown_Radius_plot', 'Mean_Crown_Volume_plot']],
                                    left_on='plot_id', right_on='plotID_x', how='left')

    # Calculate Fa
    gort_inputs = gort_inputs.merge(TEAK_plotloc_unique_df[['namedLocation', 'small_footprint_lidar_LAI']],
                                    left_on='plot_id', right_on='namedLocation', how='left')
    
    gort_inputs = gort_inputs.drop(columns=['plotID_x', 'namedLocation'])

    # gort_inputs['Fa'] = gort_inputs['small_footprint_lidar_LAI'] / (gort_inputs['Mean_Crown_Volume_plot'] * gort_inputs['Tree Density'])
    gort_inputs['Fa'] = gort_inputs['small_footprint_lidar_LAI'] / (gort_inputs['Mean_Crown_Volume_group'] * gort_inputs['Tree Density_group'])
    gort_inputs['dz'] = 0.1
    
    return gort_inputs.round(4)

def load_height_cutoffs(cutoff_csv):
    # Load the height cutoff data
    cutoffs_df = pd.read_csv(cutoff_csv)
    cutoffs_df['height_cutoff'] = cutoffs_df['height_cutoff'].fillna('No Group')
    return cutoffs_df

def output_gort_csv(gort_inputs, save_path, plot_ident):
    filename = f"gort_input_values_{plot_ident}.csv"
    gort_inputs.to_csv(os.path.join(save_path, filename), index=False)
    print(f"CSV file saved to: {os.path.join(save_path, filename)}")

def output_gort_files(gort_inputs, save_path, gort_ident):
    # Define the path to save the files
    gort_save_path = os.path.join(save_path, gort_ident)
    
    # Make sure the directory exists
    os.makedirs(gort_save_path, exist_ok=True)

    # Iterate over each unique plot_id in gort_inputs
    for plot_id in gort_inputs['plot_id'].unique():
        plot_data = gort_inputs[gort_inputs['plot_id'] == plot_id].reset_index(drop=True)  # Reset index for each plot
        group_count = plot_data['group'].nunique()  # Count the number of unique groups for this plot

        # Debugging: Check the number of rows in plot_data for the current plot_id
        print(f"Processing Plot ID: {plot_id}, Number of rows: {len(plot_data)}")

        # Define the file path for each plot
        output_file_name = os.path.join(gort_save_path, f"{plot_id}.in")

        # Open the file and start writing the data
        with open(output_file_name, 'w') as file:  # 'w' to overwrite the file (starting fresh)
            file.write(f"{group_count}\n")  # Write the number of groups at the start

            # Iterate over each group in the plot and write the corresponding line
            for idx, row in plot_data.iterrows():
                # Write the required data for each row (group)
                file.write('\t' + str(row['Tree Density_group']))  # Tree Density
                file.write('\t' + str(row['Fa']))  # Fa
                file.write('\t' + str(row['h1_group']))  # h1
                file.write('\t' + str(row['h2_group']))  # h2
                file.write('\t' + str(row['dz']))  # dz
                file.write('\t' + str(row['Mean_Crown_Radius_group']))  # Mean Crown Radius
                file.write('\t' + str(row['Mean_Vert_Crown_Radius_group']))  # Mean Vertical Crown Radius
                file.write('\t' + '0.00')  # Placeholder value
                file.write('\t' + '0.4291')  # Placeholder value
                file.write('\t' + '0.4239')  # Placeholder value
                file.write('\t' + '0.35156')  # Placeholder value
                file.write('\t' + '1')  # Placeholder value

                # Debugging: Print the group info for tracking
                print(f"Plot: {plot_id}, Group: {row['group']}, idx: {idx}, Group Count: {group_count}, h1: {row['h1_group']}")

                # Add a line break between groups if there are more than one group
                if group_count > 1 and idx < len(plot_data) - 1:
                    file.write('\n')

            # Debugging: Output a message to confirm group count and content
            print(f"Group count: {group_count}, Number of rows written: {len(plot_data)}")

        print(f"File saved: {output_file_name}")

In [162]:
cutoffs_df = load_height_cutoffs(height_cutoffs)
print(cutoffs_df)

grouped_100_df = group_trees_by_height(app_indv_100_df, cutoffs_df)
grouped_400_df = group_trees_by_height(app_indv_400_df, cutoffs_df)

gort_inputs_100_df = calculate_gort_inputs(app_indv_100_df, TEAK_plotloc_unique_df, "100", grouped_100_df)
gort_inputs_400_df = calculate_gort_inputs(app_indv_400_df, TEAK_plotloc_unique_df, "400", grouped_400_df)

output_gort_csv(gort_inputs_100_df, date_output_path, "100")
output_gort_csv(gort_inputs_400_df, date_output_path, "400")

      plotid height_cutoff
0   TEAK_001            10
1   TEAK_002            20
2   TEAK_003            17
3   TEAK_005            15
4   TEAK_006            18
5   TEAK_007      No Group
6   TEAK_010            20
7   TEAK_011            22
8   TEAK_012            20
9   TEAK_013            20
10  TEAK_014      No Group
11  TEAK_015            10
12  TEAK_016            10
13  TEAK_017        10, 20
14  TEAK_018            20
15  TEAK_019        12, 30
16  TEAK_020            13
17  TEAK_025            10
18  TEAK_043             6
19  TEAK_044        10, 19
20  TEAK_045        12, 30
21  TEAK_046        12, 27
22  TEAK_047        12, 27
Total number of trees in plot TEAK_001: 11
  Number of trees below 10.0: 6
  Number of trees above 10.0: 5
  Matched: 11
Total number of trees in plot TEAK_002: 14
  Number of trees below 20.0: 10
  Number of trees above 20.0: 4
  Matched: 14
Total number of trees in plot TEAK_003: 17
  Number of trees below 17.0: 14
  Number of trees above 17.0: 3
 

  Number of trees below 22.0: 7
  Number of trees above 22.0: 1
  Matched: 8
Total number of trees in plot TEAK_012: 41
  Number of trees below 20.0: 40
  Number of trees above 20.0: 1
  Matched: 41
Total number of trees in plot TEAK_013: 13
  Number of trees below 20.0: 7
  Number of trees above 20.0: 6
  Matched: 13
Total number of trees in plot TEAK_015: 45
  Number of trees below 10.0: 39
  Number of trees above 10.0: 6
  Matched: 45
Total number of trees in plot TEAK_016: 6
  Number of trees below 10.0: 2
  Number of trees above 10.0: 4
  Matched: 6
Total number of trees in plot TEAK_017: 12
  Number of trees below 10.0: 5
  Number of trees between 10.0 and 20.0: 4
  Number of trees above 20.0: 3
  Matched: 12
Total number of trees in plot TEAK_018: 6
  Number of trees below 20.0: 4
  Number of trees above 20.0: 2
  Matched: 6
Total number of trees in plot TEAK_019: 23
  Number of trees below 12.0: 11
  Number of trees between 12.0 and 30.0: 6
  Number of trees above 30.0: 6
  Mat

In [163]:
output_gort_files(gort_inputs_100_df, gort_date_path, "Gort_100")
output_gort_files(gort_inputs_400_df, gort_date_path, "Gort_400")

Processing Plot ID: TEAK_001, Number of rows: 2
Plot: TEAK_001, Group: Above_10.0, idx: 0, Group Count: 2, h1: 4.375
Plot: TEAK_001, Group: Below_10.0, idx: 1, Group Count: 2, h1: 1.3919
Group count: 2, Number of rows written: 2
File saved: /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_001.in
Processing Plot ID: TEAK_002, Number of rows: 2
Plot: TEAK_002, Group: Above_20.0, idx: 0, Group Count: 2, h1: 17.1753
Plot: TEAK_002, Group: Below_20.0, idx: 1, Group Count: 2, h1: 0.1271
Group count: 2, Number of rows written: 2
File saved: /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_002.in
Processing Plot ID: TEAK_003, Number of rows: 2
Plot: TEAK_003, Group: Above_17.0, idx: 0, Group Count: 2, h1: 8.0033
Plot: TEAK_003, Group: Below_17.0, idx: 1, Group Count: 2, h1: 1.805
Group count: 2, Number of rows written: 2
File saved: /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_003.in
Processing Plot ID: TEAK_005,

Plot: TEAK_043, Group: Above_6.0, idx: 0, Group Count: 2, h1: 3.287
Plot: TEAK_043, Group: Below_6.0, idx: 1, Group Count: 2, h1: 2.1453
Group count: 2, Number of rows written: 2
File saved: /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_400/TEAK_043.in
Processing Plot ID: TEAK_044, Number of rows: 3
Plot: TEAK_044, Group: 10.0_to_19.0, idx: 0, Group Count: 3, h1: 5.7781
Plot: TEAK_044, Group: Above_19.0, idx: 1, Group Count: 3, h1: 10.2612
Plot: TEAK_044, Group: Below_10.0, idx: 2, Group Count: 3, h1: 1.193
Group count: 3, Number of rows written: 3
File saved: /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_400/TEAK_044.in
Processing Plot ID: TEAK_045, Number of rows: 3
Plot: TEAK_045, Group: 12.0_to_30.0, idx: 0, Group Count: 3, h1: 6.4798
Plot: TEAK_045, Group: Above_30.0, idx: 1, Group Count: 3, h1: 21.48
Plot: TEAK_045, Group: Below_12.0, idx: 2, Group Count: 3, h1: 0.84
Group count: 3, Number of rows written: 3
File saved: /data/shared/src/STV

In [164]:
test_gort_input_dir = '/data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_11_small_footprint'

run_gort_teak(gort_input_dir, gort_output_dir, gort_process_dir)

Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_001.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_001.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_002.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_002.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100/TEAK_003.in to /data/shared/src/STV/simscenes/LidarSimulation/model/gort.in
Moved /data/shared/src/STV/simscenes/LidarSimulation/model/gort.out to /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.out/2024_11_13/Gort_100/TEAK_003.out
Copied /data/shared/src/STV/NEON_TEAK/allen/Gort/Gort.in/2024_11_13/Gort_100